<a href="https://colab.research.google.com/github/thanhhlt/CS114.O21/blob/main/Competition02/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

In [6]:
# Chuẩn bị data train, validation
def data_prep(type_score):
    df = pd.read_csv('data/' + type_score + '/train.csv')
    df = df.drop('username', axis=1)

    X = df.drop(type_score, axis=1)
    y = df[type_score]

    # Scaler
    X_scaled = StandardScaler().fit_transform(X).astype('float32')

    # Chia thành các dataset
    X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.05)

    return X_train, X_val, y_train, y_val

In [7]:
# Dự đoán tập test và lưu kết quả
def predict(model, type_score):
    df_test = pd.read_csv('data/' + type_score + '/test.csv')

    usernames = df_test['username']
    X_test = df_test.drop('username', axis=1)
    X_test = StandardScaler().fit_transform(X_test).astype('float32')

    y_pred = model.predict(X_test)
    tmp = pd.DataFrame(columns=[type_score], data=y_pred)

    result = pd.concat([usernames, tmp], axis=1)
    result.to_csv('data/' + type_score + '/' + type_score + '.csv', header=False, index=False)

In [8]:
param_grid = {'learning_rate': np.logspace(-8, -1, 8), 'depth': np.arange(6, 8), 'l2_leaf_reg': np.arange(3, 5)}

### 1. TBTL

In [9]:
X_train, X_val, y_train, y_val = data_prep('TBTL')

In [13]:
# Hyperparameter tuning với GridSearchCV
model = CatBoostRegressor(iterations=400, task_type='GPU', verbose=False)
GS = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='r2', verbose=1)
GS.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5827, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit
    train_par

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f98877920b0>,
             param_grid={'depth': array([6, 7]), 'l2_leaf_reg': array([3, 4]),
                         'learning_rate': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
             scoring='r2', verbose=1)

In [14]:
# Bộ hyerparameter tốt nhất
best_params = GS.best_params_
best_params

{'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.01}

In [15]:
# Train lại với iterations cao hơn
model = CatBoostRegressor(iterations=10000,
                          learning_rate=best_params['learning_rate'],
                          depth=best_params['depth'],
                          l2_leaf_reg=best_params['l2_leaf_reg'],
                          eval_metric='R2', early_stopping_rounds=1000,
                          verbose=100, task_type='GPU')
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0013204	test: -0.0156747	best: -0.0156747 (0)	total: 16.6ms	remaining: 2m 45s
100:	learn: 0.1347560	test: 0.0930619	best: 0.0930619 (100)	total: 1.12s	remaining: 1m 50s
200:	learn: 0.2087738	test: 0.1309405	best: 0.1309405 (200)	total: 2.04s	remaining: 1m 39s
300:	learn: 0.2688599	test: 0.1546094	best: 0.1549611 (295)	total: 3.6s	remaining: 1m 56s
400:	learn: 0.3139001	test: 0.1588005	best: 0.1621022 (362)	total: 5.8s	remaining: 2m 18s
500:	learn: 0.3483726	test: 0.1617042	best: 0.1661556 (456)	total: 8.13s	remaining: 2m 34s
600:	learn: 0.3801129	test: 0.1726449	best: 0.1744042 (561)	total: 9.05s	remaining: 2m 21s
700:	learn: 0.4096555	test: 0.1657019	best: 0.1744042 (561)	total: 9.97s	remaining: 2m 12s
800:	learn: 0.4384693	test: 0.1608025	best: 0.1744042 (561)	total: 10.9s	remaining: 2m 5s
900:	learn: 0.4633266	test: 0.1492840	best: 0.1744042 (561)	total: 11.9s	remaining: 1m 59s
1000:	learn: 0.4847625	test: 0.1356217	best: 0.1744042 (561)	total: 12.8s	remaining: 1m 55s
11

In [16]:
predict(model, 'TBTL')

### 2. TH

In [17]:
X_train, X_val, y_train, y_val = data_prep('TH')

In [18]:
model = CatBoostRegressor(iterations=400, verbose=False, task_type='GPU')
GS = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='r2', verbose=1)
GS.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5827, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit
    train_par

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f98877914b0>,
             param_grid={'depth': array([6, 7]), 'l2_leaf_reg': array([3, 4]),
                         'learning_rate': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
             scoring='r2', verbose=1)

In [19]:
best_params = GS.best_params_
best_params

{'depth': 7, 'l2_leaf_reg': 3, 'learning_rate': 0.09999999999999999}

In [20]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=best_params['learning_rate'],
                          depth=best_params['depth'],
                          l2_leaf_reg=best_params['l2_leaf_reg'],
                          eval_metric='R2', early_stopping_rounds=1000,
                          verbose=100, task_type='GPU')
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0470211	test: -0.0129980	best: -0.0129980 (0)	total: 25.9ms	remaining: 4m 19s
100:	learn: 0.6487270	test: 0.2321955	best: 0.2323326 (95)	total: 1.54s	remaining: 2m 31s
200:	learn: 0.7717937	test: 0.2490878	best: 0.2515074 (194)	total: 3.64s	remaining: 2m 57s
300:	learn: 0.8223239	test: 0.2649641	best: 0.2687124 (287)	total: 5.8s	remaining: 3m 7s
400:	learn: 0.8596799	test: 0.2656056	best: 0.2690517 (359)	total: 7.71s	remaining: 3m 4s
500:	learn: 0.8833988	test: 0.2488453	best: 0.2690517 (359)	total: 9.07s	remaining: 2m 52s
600:	learn: 0.8961306	test: 0.2339485	best: 0.2690517 (359)	total: 10.4s	remaining: 2m 42s
700:	learn: 0.9084778	test: 0.2291392	best: 0.2690517 (359)	total: 11.7s	remaining: 2m 35s
800:	learn: 0.9164366	test: 0.2176992	best: 0.2690517 (359)	total: 13s	remaining: 2m 29s
900:	learn: 0.9237798	test: 0.2127016	best: 0.2690517 (359)	total: 14.4s	remaining: 2m 24s
1000:	learn: 0.9288357	test: 0.2055239	best: 0.2690517 (359)	total: 15.7s	remaining: 2m 21s
1100:

In [21]:
predict(model, 'TH')

### 3. QT

In [23]:
X_train, X_val, y_train, y_val = data_prep('QT')

In [24]:
model = CatBoostRegressor(iterations=400, verbose=False, task_type='GPU')
GS = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='r2', verbose=1)
GS.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5827, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit
    train_par

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f9887792770>,
             param_grid={'depth': array([6, 7]), 'l2_leaf_reg': array([3, 4]),
                         'learning_rate': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
             scoring='r2', verbose=1)

In [25]:
best_params = GS.best_params_
best_params

{'depth': 7, 'l2_leaf_reg': 4, 'learning_rate': 0.09999999999999999}

In [26]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=best_params['learning_rate'],
                          depth=best_params['depth'],
                          l2_leaf_reg=best_params['l2_leaf_reg'],
                          eval_metric='R2', early_stopping_rounds=1000,
                          verbose=100, task_type='GPU')
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0232699	test: -0.0071321	best: -0.0071321 (0)	total: 25.2ms	remaining: 4m 11s
100:	learn: 0.5261388	test: -0.0548198	best: 0.0206299 (6)	total: 1.44s	remaining: 2m 20s
200:	learn: 0.6762425	test: -0.0525657	best: 0.0206299 (6)	total: 2.79s	remaining: 2m 15s
300:	learn: 0.7567838	test: -0.0503788	best: 0.0206299 (6)	total: 5.72s	remaining: 3m 4s
400:	learn: 0.7983617	test: -0.0621824	best: 0.0206299 (6)	total: 8.11s	remaining: 3m 14s
500:	learn: 0.8205678	test: -0.0722676	best: 0.0206299 (6)	total: 9.42s	remaining: 2m 58s
600:	learn: 0.8368427	test: -0.0746081	best: 0.0206299 (6)	total: 10.8s	remaining: 2m 48s
700:	learn: 0.8490583	test: -0.0755384	best: 0.0206299 (6)	total: 12.1s	remaining: 2m 40s
800:	learn: 0.8637323	test: -0.0722407	best: 0.0206299 (6)	total: 13.5s	remaining: 2m 34s
900:	learn: 0.8744123	test: -0.0747580	best: 0.0206299 (6)	total: 14.8s	remaining: 2m 29s
1000:	learn: 0.8840183	test: -0.0683475	best: 0.0206299 (6)	total: 16.1s	remaining: 2m 24s
bestTest =

In [27]:
predict(model, 'QT')

### 4. CK

In [28]:
X_train, X_val, y_train, y_val = data_prep('CK')

In [29]:
model = CatBoostRegressor(iterations=400, verbose=False, task_type='GPU')
GS = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='r2', verbose=1)
GS.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5827, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit
    train_par

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f98702eb550>,
             param_grid={'depth': array([6, 7]), 'l2_leaf_reg': array([3, 4]),
                         'learning_rate': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
             scoring='r2', verbose=1)

In [30]:
best_params = GS.best_params_
best_params

{'depth': 7, 'l2_leaf_reg': 3, 'learning_rate': 0.01}

In [31]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=best_params['learning_rate'],
                          depth=best_params['depth'],
                          l2_leaf_reg=best_params['l2_leaf_reg'],
                          eval_metric='R2', early_stopping_rounds=1000,
                          verbose=100, task_type='GPU')
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0034791	test: -0.0466221	best: -0.0466221 (0)	total: 25.1ms	remaining: 4m 10s
100:	learn: 0.1926743	test: 0.0756151	best: 0.0756151 (100)	total: 1.46s	remaining: 2m 23s
200:	learn: 0.2897506	test: 0.1251442	best: 0.1251442 (200)	total: 2.77s	remaining: 2m 14s
300:	learn: 0.3588620	test: 0.1577314	best: 0.1578408 (299)	total: 4.03s	remaining: 2m 9s
400:	learn: 0.4072325	test: 0.1845837	best: 0.1845837 (400)	total: 5.33s	remaining: 2m 7s
500:	learn: 0.4475313	test: 0.2053096	best: 0.2053096 (500)	total: 6.59s	remaining: 2m 4s
600:	learn: 0.4810151	test: 0.2206822	best: 0.2206822 (600)	total: 7.81s	remaining: 2m 2s
700:	learn: 0.5117178	test: 0.2356344	best: 0.2357028 (699)	total: 9.59s	remaining: 2m 7s
800:	learn: 0.5390710	test: 0.2491566	best: 0.2504056 (794)	total: 12.8s	remaining: 2m 26s
900:	learn: 0.5608435	test: 0.2549666	best: 0.2549666 (900)	total: 14.1s	remaining: 2m 22s
1000:	learn: 0.5839723	test: 0.2626168	best: 0.2628097 (981)	total: 15.5s	remaining: 2m 18s
1100

In [32]:
predict(model, 'CK')